<a href="https://colab.research.google.com/github/ayush-u01/BTP-STOCK/blob/main/FinBERT_Sentiment_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
import pandas

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')

Ignore these tokens = tokenizer... lines, they are just for testing

In [ ]:
tokens = tokenizer.encode('The stock is rallying due to surge in the Sensex.', return_tensors='pt')

In [ ]:
tokens = tokenizer.encode('Terrible Stock with low yields. The recession is negatively affecting the returns.', return_tensors='pt')

In [ ]:
tokens = tokenizer.encode('Very good performaning rising gaining stock', return_tensors='pt')

In [ ]:
tokens = tokenizer.encode('This felt terrific. Truly spectacular.', return_tensors = 'pt')

In [ ]:
tokens = tokenizer.encode('Terrible. Aweful. What a waste. This is a bad stock', return_tensors = 'pt')

In [ ]:
tokens = tokenizer.encode('Okay Okay. Medium Performace. Not good not bad.', return_tensors = 'pt')

In [ ]:
tokens = tokenizer.encode('Neutral Bad', return_tensors = 'pt')

In [ ]:
tokens = tokenizer.encode('What an underperforming asset.', return_tensors='pt')

Reading the csv files using pandas

In [ ]:
df = pandas.read_csv('apple-twitter-sentiment-texts.csv')

In [ ]:
df = pandas.read_csv('dataforproject.csv')

In [ ]:
df = pandas.read_csv('30.csv')

For checking if correct csv file opened or not

In [ ]:
df.shape

(246, 13)

No need to run these few lines given below. Just for previewing the csv file

In [ ]:
df.head()

In [ ]:
df.sample

In [ ]:
df['text'].head

In [ ]:
df['Content'].head

In [ ]:
df.iloc[5]

In [ ]:
df['Content'].iloc[500]

Function for tokenization and calculating the sentiment score.

In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

Ignore this function

In [ ]:
def map_sentiment(rating):
  if (rating < 3):
    return int(-1)
  elif (rating == 3):
    return int(0)
  else:
    return int(1)

This one is correct

In [ ]:
def map_sentiment(rating):
  if (rating == 1):
    return int(1)
  elif (rating == 2):
    return int(-1)
  else:
    return int(0)

Ignore, just for testing

In [ ]:
map_sentiment(sentiment_score(df['text'].iloc[2]))

In [ ]:
map_sentiment(sentiment_score(df['Content'].iloc[500]))

In [ ]:
map_sentiment(sentiment_score('completely bullish for tomorrow. my prediction says a strong upside move can come. I might be wrong but wont short anything tomorrow. dont carry anything over night. #nifty #nifty50 #banknifty #StockMarket'))

In [ ]:
map_sentiment(sentiment_score('good stock'))

In [ ]:
map_sentiment(sentiment_score('Terrible Mistake. Bad Show. Poor Review. Wastage. Aweful'))

Ignore this as well

In [ ]:
df['Content'] = df['Content'].astype(str)

In [ ]:
df['prediction'] = df['text'].apply(lambda x: map_sentiment(sentiment_score(x[:512])))

Ignore this

In [ ]:
df['prediction'][500] = 'okay'

In [ ]:
df.head()

In [ ]:
df

Cleaning the data.
Deleting tweets with no text data.
Also deleting tweets with no likes

In [ ]:
df = df.dropna(subset=['Content'])

In [ ]:
df.drop(df[df['Likes'] > 0].index, inplace=True)

To see how much data left after some tweets deleted

In [ ]:
df.shape

(109, 13)

For each tweet (row in csv file), a prediction (sentiment) must be calculated. So a prediction column is being created to be filled with sentiment prediction from Tweet's content column.

In [ ]:
df['prediction'] = df['Content'].apply(lambda x: map_sentiment(sentiment_score(x[:512])))

Ignore

In [ ]:
(df['Content'].iloc[2])

This is the real result. It shows number of tweets with neutral, negative and positive sentiment.

In [ ]:
df['prediction'].value_counts()

 0    95
-1    10
 1     4
Name: prediction, dtype: int64

Ignore all given below

In [ ]:
tokens

tensor([[  101,  6659,  4518,  2007,  2659, 16189,  1012,  1996, 19396,  2003,
         19762, 12473,  1996,  5651,  1012,   102]])

In [ ]:
result = model(tokens)

In [ ]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.8501,  2.7124, -0.3291]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
result.logits

tensor([[-1.8501,  2.7124, -0.3291]], grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

2